# Exploring Object-Oriented Multiprocessing Using MNIST Dataset

## Import MNIST Data

Source:  http://yann.lecun.com/exdb/mnist/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mnist
from datetime import datetime

# Import images and labels
images_train = mnist.train_images()
images_train_labels = mnist.train_labels()
images_test = mnist.test_images()
images_test_labels = mnist.test_labels()

# Scale testing and training images
images_test = images_test/255
images_train = images_train/255

print("Dimensions of training data:", images_train.shape)
print("Dimensions of training labels:", images_train_labels.shape, "\n")
print("Dimensions of test images:", images_test.shape)
print("Dimensions of test labels:", images_test_labels.shape)

Dimensions of training data: (60000, 28, 28)
Dimensions of training labels: (60000,) 

Dimensions of test images: (10000, 28, 28)
Dimensions of test labels: (10000,)


In [2]:
import numpy as np
import logging

class KnnClassifier(object):
    @staticmethod   
    def euclidean(x,y):
        """
        Inputput
            x: a 1-dimensional numpy array
            y: a 1-dimensional numpy array
        Output
            A single number, the euclidean distance between x and y
        """      

        return(np.sqrt(np.sum((x - y)**2)))
    
    @staticmethod
    def many_distances(x,Y):

        """
        Input
            x: a single point as a numpy array
            Y: a 2-dimensional numpy array
        Output
            a numpy array of euclidean distances
        """
        result = np.zeros(Y.shape[0])
        for idx in range(0, Y.shape[0]):            
            dist = KnnClassifier.euclidean(x, Y[idx])          
            result[idx] = dist  
        return(result)
    
    @staticmethod
    def closest_indices(dists, n):

        """
        Input
            dists: a numpy array of distances (1 dimensional)
            n: the number of lowest values to find
        Output
            a numpy array with the indexes in dists where the
            n lowest values are located.
        """

        arrIndexes = np.argsort(dists)   
        return(arrIndexes[0:n])
    
    @staticmethod
    def get_values_by_index(indices, values):
        """
        Input
            indices: a numpy array of indices
            values: a list of values
        Output
            a list of elements from values whose indexes
            are the values in indices
        """
        arr = np.array(values)
        return(arr[indices])
    
    @staticmethod
    def get_mode(values):
        """
        Input
            values: a lists of values
        Output
            the most common value in the list.
            If there's a tie, break it any way you want to.
        """

        counts = np.unique(values, return_counts=True)
        cnt = np.argsort(counts[1])   
        colorIndex = cnt[len(cnt) -1]
        label = counts[0][colorIndex]   
        return(label)
    
    @staticmethod
    def knn(ind, test_pts, train_pts, labels, k):
        """
        Input
            test: A 2-D numpy array of points (rows) and features (cols)
            train: A 2-D numpy array of points (rows) and features (cols)
            labels: A list of labels associated with train points
        Output
            A list of best guesses for the test labels
        """      
        output = []    
        startIdx = ind[0]
        endIdx = ind[1]
        subsetOfPoints = test_pts[startIdx:endIdx]
       
        for i in range(0,subsetOfPoints.shape[0]):                     
            dists = KnnClassifier.many_distances(subsetOfPoints[i],train_pts)              
            label_indices = KnnClassifier.closest_indices(dists, k)           
            labelSet = KnnClassifier.get_values_by_index(label_indices, labels)          
            predictedLabel = KnnClassifier.get_mode(labelSet)
            output.append(predictedLabel)      
            
        
        return output
    
    @staticmethod    
    def multiclass_accuracy(truth,predictions):
        """
        Input
        truth: a list of true labels
        predictions: a list of predicted labels
        Output
        a single value - the multiclass accuracy
        """
        cmp = np.array(truth) == np.array(predictions)
        wrong = len(cmp[cmp == False])
        right = len(cmp[cmp == True])
        accuracy = right/(wrong + right)
        return(accuracy)

## Utilize ProcessPoolExecutor
Source:  https://docs.python.org/3/library/concurrent.futures.html

In [3]:
import concurrent.futures
# import math

class MultiProcess(object):
    
    def __init__(self, testingImages, testingLabels, trainingImages, trainingLabels, kSize):
        self.start_time = None
        MultiProcess.testingImages = testingImages
        MultiProcess.testingLabels = testingLabels
        MultiProcess.trainingImages = trainingImages
        MultiProcess.trainingLabels = trainingLabels
        MultiProcess.kSize = kSize       
        
    @staticmethod 
    def testMethod(item):       
        intermediateResult = KnnClassifier.knn(item, 
                                                   MultiProcess.testingImages, 
                                                   MultiProcess.trainingImages, 
                                                   MultiProcess.trainingLabels, 
                                                   MultiProcess.kSize)
        return intermediateResult

    def __main(self, tupList, numProcesses):       
        self.start_time = datetime.now()
        finalResult = []        
        with concurrent.futures.ProcessPoolExecutor(max_workers = numProcesses) as executor:
            for tupRange, label in zip(tupList, executor.map(MultiProcess.testMethod, tupList, timeout=None, chunksize=1)):               
                for item in label:                    
                    finalResult.append(item)
        return finalResult
            
    def __getInput(self, size, interval):    
        arr = []       
        remainder = size % interval
        iterations = int(size/interval)
        for i in range(0,iterations):
            start = i*interval
            end = start + interval        
            arr.append((start, end)) 
        if(remainder != 0):
            arr.append((end, end + remainder))       
        return arr

    def invokeMultiProcessMethod(self, numProcesses, intervalSize):       
        maxNum = MultiProcess.testingImages.shape[0]       
        tupList = self.__getInput(maxNum, intervalSize)
        print("Input work items:  ", tupList)        
        predictedLabels = self.__main(tupList, numProcesses)
        self.__reportResults(predictedLabels, maxNum, numProcesses)
        #return predictedLabels    
    
    def __reportResults(self, predictedLabels, maxNum, numProcesses):
        print("Predicted Labels:  ", predictedLabels)
        end_time = datetime.now()
        diff = end_time - self.start_time
        print("Test Image Set Size: ", maxNum)
        print("Number of Processes:", numProcesses)
        print("Start Time =", 
              self.start_time.strftime('%H:%M:%S.%f'), 
              "\nEnd Time =", end_time.strftime('%H:%M:%S.%f'), 
              "\nTotal Run Time (Seconds) =", 
              diff.total_seconds())
        accuracy = KnnClassifier.multiclass_accuracy(MultiProcess.testingLabels, predictedLabels)
        print("Classifier Accuracy (k = 1):",'{:.1%}'.format(accuracy), "\n")        

In [4]:
myTestSize = 200
multiProcess1 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess1.invokeMultiProcessMethod(numProcesses=1, intervalSize=2)

multiProcess2 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess2.invokeMultiProcessMethod(numProcesses=2, intervalSize=2)

multiProcess3 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess3.invokeMultiProcessMethod(numProcesses=3, intervalSize=2)

multiProcess4 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess4.invokeMultiProcessMethod(numProcesses=4, intervalSize=2)

multiProcess5 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess5.invokeMultiProcessMethod(numProcesses=5, intervalSize=2)

multiProcess6 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess6.invokeMultiProcessMethod(numProcesses=6, intervalSize=2)

multiProcess7 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess7.invokeMultiProcessMethod(numProcesses=7, intervalSize=2)

multiProcess8 = MultiProcess(testingImages=images_test[:myTestSize], 
                            testingLabels=images_test_labels[:myTestSize],
                            trainingImages=images_train, 
                            trainingLabels=images_train_labels, 
                            kSize = 1)
multiProcess8.invokeMultiProcessMethod(numProcesses=8, intervalSize=2)

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 1

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 1 <br>
Start Time = 01:03:06.029583 <br>
End Time = 01:08:25.373788 <br>
Total Run Time (Seconds) = 319.344205 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 2 <br>
Start Time = 01:08:25.380227 <br>
End Time = 01:11:14.553347 <br>
Total Run Time (Seconds) = 169.17312 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 3 <br>
Start Time = 01:11:14.559156 <br>
End Time = 01:13:15.373990  <br>
Total Run Time (Seconds) = 120.814834 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 4 <br>
Start Time = 01:13:15.378757 <br>
End Time = 01:15:04.279360 <br>
Total Run Time (Seconds) = 108.900603 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 5 <br>
Start Time = 01:15:04.290049 <br>
End Time = 01:16:48.587514 <br>
Total Run Time (Seconds) = 104.297465 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 6 <br>
Start Time = 01:16:48.599535 <br>
End Time = 01:18:16.806794 <br>
Total Run Time (Seconds) = 88.207259 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 7 <br>
Start Time = 01:18:16.814265 <br>
End Time = 01:19:34.582989 <br>
Total Run Time (Seconds) = 77.768724 <br>
Classifier Accuracy (k = 1): 99.0% <br>

Input work items:   [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30), (30, 32), (32, 34), (34, 36), (36, 38), (38, 40), (40, 42), (42, 44), (44, 46), (46, 48), (48, 50), (50, 52), (52, 54), (54, 56), (56, 58), (58, 60), (60, 62), (62, 64), (64, 66), (66, 68), (68, 70), (70, 72), (72, 74), (74, 76), (76, 78), (78, 80), (80, 82), (82, 84), (84, 86), (86, 88), (88, 90), (90, 92), (92, 94), (94, 96), (96, 98), (98, 100), (100, 102), (102, 104), (104, 106), (106, 108), (108, 110), (110, 112), (112, 114), (114, 116), (116, 118), (118, 120), (120, 122), (122, 124), (124, 126), (126, 128), (128, 130), (130, 132), (132, 134), (134, 136), (136, 138), (138, 140), (140, 142), (142, 144), (144, 146), (146, 148), (148, 150), (150, 152), (152, 154), (154, 156), (156, 158), (158, 160), (160, 162), (162, 164), (164, 166), (166, 168), (168, 170), (170, 172), (172, 174), (174, 176), (176, 178), (178, 180), (180, 182), (182, 184), (184, 186), (186, 188), (188, 190), (190, 192), (192, 194), (194, 196), (196, 198), (198, 200)] <br>
Predicted Labels:   [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 9, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 5, 1, 6, 4, 2] <br>
Test Image Set Size:  200 <br>
Number of Processes: 8 <br>
Start Time = 01:19:34.585714 <br>
End Time = 01:20:43.092921  <br>
Total Run Time (Seconds) = 68.507207 <br>
Classifier Accuracy (k = 1): 99.0% <br>